In [1]:
import pandas as pd
import os
import numpy as np
import folium
import h3
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
import branca

p = os.path.join
cmap = plt.get_cmap('hot_r')
df = pd.read_csv(p('..', '03 Prediction', 'final_prediction.csv'), index_col=0)
    
if not os.path.exists('www/maps'):
    os.mkdir('www/maps')

In [2]:
def normalize_name(city_name):
    return city_name.lower().replace(' ', '')

# Maps

In [3]:
for (c, city_df) in df.groupby('city'):
    ring = {r for h in city_df.index for r in h3.k_ring(h, 3)}
    latlons = [h3.h3_to_geo(r) for r in ring]
    lat, lon = np.array(latlons).mean(axis=0)
    m = folium.Map(location=[lat, lon])
    m.fit_bounds(latlons)
    cmap_legend = branca.colormap.LinearColormap([cmap(i/10) for i in range(10)], vmin=city_df['NO2_adjusted'].min(), vmax=city_df['NO2_adjusted'].max())
    cmap_legend.caption = 'NO2 Level (ppb)'
    cmap_legend.add_to(m)

    for r in ring:
        try:
            level = city_df.loc[r]['NO2_adjusted']
        except KeyError:
            continue
        folium.Polygon(
            h3.h3_to_geo_boundary(r),
            tooltip=f'{np.round(level, 1)} ppb',
            fill_color= rgb2hex(cmap(level/city_df['NO2_adjusted'].max())), 
            color='black',
            fill_opacity= 0.6
        ).add_to(m)
    m.save(p('www','maps',f'{normalize_name(c)}.html'))

In [4]:
cities = [('index', 'All')] + [(normalize_name(c), c) for c in df.city.unique()]

# Pages

In [5]:
from jinja2 import FileSystemLoader, Environment

def render_from_template(directory, template_name, **kwargs):
    loader = FileSystemLoader(directory)
    env = Environment(loader=loader)
    template = env.get_template(template_name)
    return template.render(**kwargs)

for (c, _) in cities:
    with open(p('www', f'{c}.html'), 'w') as f:
        f.write(render_from_template('.', 'template.jinja', current=c, cities=cities))